# ДЗ-1 Chat classification
## Финтех школа зима 2019
### Усеинов Булат 

In [1]:
import pandas as pd
import numpy as np
import re
import razdel 
import tqdm
import maru
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from random import choices, seed
from sklearn.feature_extraction.text import TfidfVectorizer
import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.utils import resample

/Users/bulat/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/bulat/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/bulat/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/Users/bulat/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


- 0 - чат о Python
- 1 - чат знакомств
- 2 - чат про ML

In [2]:
seed(45)

# Загрузим данные

In [3]:
data = pd.read_csv("/Users/bulat/Desktop/тинькофф/01_dz/tfstextclassification/train.csv", ",")
data.head()

,id,text,label
0,0,ж 44 позн с мужч от 40 лет 0678935919,1
1,1,уже и не вспомню где это увидел,2
2,2,", sep=';', decimal=',' может изза этого",2
3,3,НУЖНА ДЕВ ДЛЯ С\\О -19л.0637048428смс,1
4,4,.М54.ИЩУ Д.Ж. С-П.ДЛЯ ЛАСК...НЕКМ.0508665030....,1


In [4]:
data.shape

(112467, 3)

In [5]:
#столбец id совсем бесполезное, его можно спокойно удалять
data = data.drop('id', axis = 1)
data.head()

,text,label
0,ж 44 позн с мужч от 40 лет 0678935919,1
1,уже и не вспомню где это увидел,2
2,", sep=';', decimal=',' может изза этого",2
3,НУЖНА ДЕВ ДЛЯ С\\О -19л.0637048428смс,1
4,.М54.ИЩУ Д.Ж. С-П.ДЛЯ ЛАСК...НЕКМ.0508665030....,1


In [6]:
print('Python:',data[data['label']==0].shape[0],
      'Знакомства:',data[data['label']==1].shape[0], 
      'ML:', data[data['label']==2].shape[0])

Python: 21616 Знакомства: 50000 ML: 40851


### Заметим, что классы не сбалансированы, поэтому сбалансируем их

In [7]:
a = np.array([21616, 50000, 40851]).mean()
a - 21616

15873.0

In [8]:
b = resample(data[data.label == 0], replace =True, n_samples = 15873, random_state = 42)

In [9]:
data = pd.concat([data, b])

In [10]:
data.shape

(128340, 2)

## Выполним препроцессинг 
(Далее оказалось, что выполнять все шаги препроцессинга - ухудшать точность модели в данной задачи,пожэтому использовался метод из scikit с использованием собственного препроцессинга)

In [11]:
specials = punctuation + "«»—“-№)"
pattern = re.compile("[" + re.escape(specials) + "]")

stopRus = stopwords.words('russian')
stopEng = stopwords.words('english')

stemmer_rus = SnowballStemmer("russian")
stemmer_eng = SnowballStemmer("english")

In [12]:
def tokkenize_razdel(text):
    return [token.text for token in razdel.tokenize(text)]

maru_r = maru.get_analyzer('rnn')

def lemmatize_maru(tokens):
    return [morph.lemma for morph in maru_r.analyze(tokens)]

def not_consists_digit(word):
    for symb in word:
        if symb.isdigit():
            return False
    return True

def preprocessing(text):
    text = text.lower() # приводим к нижнему регистру
    text = re.sub(pattern, ' ', text) # удаляем все специальные символы в том числе знаки препинания
    text = text.replace('ё', 'е') # заменяем ё на е
    
    text_tokens = tokkenize_razdel(text) # токкенизация 

    text_tokens = lemmatize_maru(text_tokens) # лемматизация
    
    filter1 = list(map(lambda x: not_consists_digit(x), text_tokens)) # удаление чисел
    text_tokens = [i for indx,i in enumerate(text_tokens) if filter1[indx] == True]
    filter2 = list(map(lambda x: x not in (stopRus and stopEng), text_tokens)) # удаление стоп-слов
    text_tokens = [i for indx,i in enumerate(text_tokens) if filter2[indx] == True]

    return text_tokens

In [13]:
target = data['label']
data = data.drop('label', axis = 1)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.01, random_state=42)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

idf_vectorizer = TfidfVectorizer(tokenizer=tokkenize_razdel, stop_words = stopRus)

In [43]:
def ev_vectorizer(vectorizer):
    X = vectorizer.fit_transform(tqdm.tqdm_notebook(X_train.text, desc='Vectorizing train:'))

    model = LinearSVC(random_state=42)
    model.fit(X, y_train)
    
    X_test1 = vectorizer.transform(tqdm.tqdm_notebook(test.text, desc='Vectorizing test:'))
    predictions = model.predict(X_test1)

    return predictions

In [44]:
pred = ev_vectorizer(idf_vectorizer)

In [45]:
test = pd.read_csv("/Users/bulat/Desktop/тинькофф/01_dz/tfstextclassification/test.csv", ",")

In [46]:
test = test.drop('id', axis = 1)

In [47]:
df = pd.DataFrame(columns=['label'])
df['label'] = pred

In [48]:
df = df.rename_axis("id", axis = 0)

In [51]:
df.head()

,label
id,
0,2
1,0
2,2
3,2
4,1


In [52]:
df.to_csv('predictions.csv')